# STA API time-travel extension
This extension assists istSTA users in accessing historical time travel data. It enables users to retrieve information from a web service as it appeared at a specific moment in time, using a new query parameter called **as_of**.

Additionally, it introduces a new entity called Commit, which enables data lineage, allowing users to trace data changes. 
From a scientific perspective, this extension enables FAIR data management by allowing datasets to be permanently cited. This is achieved by using a combination of the service address (<font color='red'>in red</font>), the request that returns the dataset (<font color='green'>in green</font>), and the dataset's status at a specific time instant (<font color='orange'>in orange</font>) as a Persistent Identifier for reference.

Example: <font color='red'>https://&lt;base_url&gt;/&lt;version&gt;/</font><font color='green'>&lt;entity&gt;?$expand=&lt;entity&gt;</font><font color='orange'>&\$as_of=&lt;date_time&gt;</font>

## Definition
The *time-travel* extension adds the following optional query parameters to any STA request:

| Parameter | Type               | Description                                                                       |
| --------- | ------------------ | --------------------------------------------------------------------------------- |
| *as_of*   | ISO 8601 date-time | a date-time parameter to specify the exact moment for which the data is requested |
| *from_to* | ISO 8601 period    | a period parameter to specify the time interval for which the data is requested   |

The *time-travel* extension introduces a new entity, Commit, with the following properties:

| Properties     | Type               | Multiplicity and use | Description                                                                    |
| -------------- | ------------------ | -------------------- | ------------------------------------------------------------------------------ |
| *author*       | string(128)        | One (mandatory)      | Authority, Username or User Profile Link                                       |
| *encodingType* | string             | One (optional)       | The encoding type of the message (default is `text`).                          |
| *message*      | string(256)        | One (mandatory)      | Commit message detailing the scope, motivation, and method of the transaction. |
| *date*         | ISO 8601 date-time | One (mandatory)      | A date-time that specifies the exact moment when the commit was executed.      |

Commits are related to SensorThings API entities with a one-to-zero-or-one (1:0..1) relationship.

### Preliminary Steps

This section contains the preliminary steps to set up the base URL, headers, and import necessary libraries.

In [ ]:
import requests
import json
import re
import istsos4_utils as st
from IPython.display import display, Markdown
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

# base url
base_url = "http://api:5000/istsos4/v1.1/"

# Headers (optional, but specifying Content-Type ensures proper handling of JSON data)
headers = {
    'Content-Type': 'application/json'
}

meteo = st.sta(base_url)

### Creating multiple related entities with deep insert

In [ ]:
body = {
    "description": "Water level, water temperature and water electrical conductivity recorder Ticino river",
    "name": "FIU_VAL",
    "properties": {
        "keywords": "water,river,height,temperature,conductivity,ACSOT",
        "description": "River level, water temperature and water electrical conductivity fiume Ticino valle"
    },
    "Locations": [
        {
            "description": "",
            "name": "fiume Ticino valle",
            "location": {
                "type": "Point",
                "coordinates": [
                    8.956099,
                    46.172245
                ]
            },
            "encodingType": "application/vnd.geo+json"
        }
    ],
    "Datastreams": [
        {
            "unitOfMeasurement": {
                "name": "Celsius degree",
                "symbol": "°C",
                "definition": ""
            },
            "description": "",
            "name": "WT_FIU_VAL",
            "observationType": "",
            "ObservedProperty": {
                "name": "ground:water:temperature",
                "definition": "",
                "description": "Ground water temperature"
            },
            "Sensor": {
                "description": "",
                "name": "Ecolog 1000",
                "encodingType": "application/json",
                "metadata": '{"brand": "OTT", "type": "Pressure, temperature, electrical conductivity sensor"}'
            },
            "Observations": [
                { "result": 23.7, "phenomenonTime": "2024-09-27T17:00:00", "resultQuality": "100"},
                { "result": 23.7, "phenomenonTime": "2024-09-27T17:10:00", "resultQuality": "100"},
                { "result": 23.7, "phenomenonTime": "2024-09-27T17:20:00", "resultQuality": "100"},
                { "result": 23.7, "phenomenonTime": "2024-09-27T17:30:00", "resultQuality": "100"},
                { "result": 23.7, "phenomenonTime": "2024-09-27T17:40:00", "resultQuality": "100"},
                { "result": 23.7, "phenomenonTime": "2024-09-27T17:50:00", "resultQuality": "100"},
                { "result": 23.7, "phenomenonTime": "2024-09-27T18:00:00", "resultQuality": "100"},
                { "result": 32.8, "phenomenonTime": "2024-09-27T18:10:00", "resultQuality": "313"},
                { "result": 23.8, "phenomenonTime": "2024-09-27T18:20:00", "resultQuality": "100"},
                { "result": 23.8, "phenomenonTime": "2024-09-27T18:30:00", "resultQuality": "100"},
                { "result": 23.8, "phenomenonTime": "2024-09-27T18:40:00", "resultQuality": "100"},
                { "result": 23.8, "phenomenonTime": "2024-09-27T18:50:00", "resultQuality": "100"},
                { "result": 23.8, "phenomenonTime": "2024-09-27T19:00:00", "resultQuality": "100"},
                { "result": 23.8, "phenomenonTime": "2024-09-27T19:10:00", "resultQuality": "100"},
                { "result": 23.8, "phenomenonTime": "2024-09-27T19:20:00", "resultQuality": "100"},
                { "result": 23.8, "phenomenonTime": "2024-09-27T19:30:00", "resultQuality": "100"},
                { "result": 23.8, "phenomenonTime": "2024-09-27T19:40:00", "resultQuality": "100"},
                { "result": 23.8, "phenomenonTime": "2024-09-27T19:50:00", "resultQuality": "100"},
                { "result": 23.8, "phenomenonTime": "2024-09-27T20:00:00", "resultQuality": "100"},
                { "result": 23.8, "phenomenonTime": "2024-09-27T20:10:00", "resultQuality": "100"},
            ]
        }
    ]
}

# POST request with the JSON body
response = requests.post(base_url + 'Things', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Thing created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Thing @iot.id using a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    thing_id = int(match.group(1))
else:
    print("No number found in parentheses.")

### Update with time-travel extension

#### Retrieve Thing (With the as_of value set to the date of the request)

In [ ]:
response = requests.get(f"{base_url}Things({thing_id})")
json_data = json.dumps(response.json(), indent=2)
md = f"```json\n{json_data}\n```"
display(Markdown(md))

#### Update Thing

In [ ]:
body = {
    "description": "(UPDATED) Water level, water temperature and water electrical conductivity recorder Ticino river",
    "name": "(UPDATED) FIU_VAL",
    "Commit": {
        "author": "YOUR NAME",
        "message": "update name and description",
    }
}

# POST request with the JSON body
response = requests.patch(base_url + f"Things({thing_id})", data=json.dumps(body))

# Check if the request was successful (status code 2xx)
if response.status_code == 200:
    print(f"Thing {thing_id} updated successfully")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

datetime_update = datetime.now() - timedelta(seconds=1)
datetime_update = datetime_update.strftime("%Y-%m-%dT%H:%M:%S.%f")

#### Retrieve Thing (With the as_of value set to the date of the request)

In [ ]:
response = requests.get(f"{base_url}Things({thing_id})?$expand=Commit")
json_data = json.dumps(response.json(), indent=2)
md = f"```json\n{json_data}\n```"
display(Markdown(md))

#### Retrieve Thing at a specific instant (with the as_of value set to one second prior to the update date)

In [ ]:
response = requests.get(f"{base_url}Things({thing_id})?$expand=Commit&$as_of={datetime_update}")
json_data = json.dumps(response.json(), indent=2)
md = f"```json\n{json_data}\n```"
display(Markdown(md))

#### Plot Observations before update

In [ ]:
response = requests.get(f"{base_url}Thing({thing_id})/Datastreams/Observations")
json_data = json.dumps(response.json(), indent=2)

# Parse the JSON data
data = json.loads(json_data)

# Extract the phenomenon times and results
phenomenon_times = [datetime.strptime(obs["phenomenonTime"], "%Y-%m-%dT%H:%M:%S%z") for obs in data["value"]]
results = [obs["result"] for obs in data["value"]]

# Plotting the results
plt.figure(figsize=(12, 7))  # Increase figure size for better visibility

# Customize the plot with better aesthetics
plt.plot(phenomenon_times, results, marker='o', markersize=8, linestyle='-', color='#1D4E89', label="Water Temperature", linewidth=2)

# Formatting the x-axis to show dates and times properly
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=1))
plt.gcf().autofmt_xdate()

# Adding a grid with better styling
plt.grid(True, which='both', linestyle='--', linewidth=0.6, color='gray', alpha=0.7)

# Adding labels and title with enhanced fonts
plt.title('Water Temperature Over Time', fontsize=16, fontweight='bold', color='#264653')
plt.xlabel('Phenomenon Time', fontsize=14, fontweight='bold', color='#264653')
plt.ylabel('Temperature (°C)', fontsize=14, fontweight='bold', color='#264653')

# Adding a legend
plt.legend(loc='upper left', fontsize=12)

# Adding custom ticks on the y-axis
plt.yticks(fontsize=12)

# Tight layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()

#### Retrieve Observation outlier (Interquartile Range method)

In [ ]:
# Convert the results (temperatures) to a NumPy array for calculations
results = np.array([obs["result"] for obs in data["value"]])

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = np.percentile(results, 25)
Q3 = np.percentile(results, 75)

# Compute the Interquartile Range (IQR)
IQR = Q3 - Q1

# Define the bounds for detecting outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify the outliers
outliers_info = [(i, obs["@iot.id"], obs["result"]) for i, obs in enumerate(data["value"]) if obs["result"] < lower_bound or obs["result"] > upper_bound]
outliers = [(info[1], info[2]) for info in outliers_info]
outlier_indices = [info[0] for info in outliers_info]

# Display the IDs and corresponding outlier values
print("Outliers (ID, Temperature):", outliers)
observation_id = outliers[0][0]

#### Retrieve Observation outlier (Z-score method)

In [ ]:
# Convert the results (temperatures) to a NumPy array for calculations
results = np.array([obs["result"] for obs in data["value"]])

# Calculate mean and standard deviation
mean = np.mean(results)
std_dev = np.std(results)

# Calculate z-scores
z_scores = (results - mean) / std_dev

# Define the threshold for identifying outliers
threshold = 3

# Identify the outliers
outliers_info = [(i, obs["@iot.id"], obs["result"]) for i, (obs, z) in enumerate(zip(data["value"], z_scores)) if abs(z) > threshold]
outliers = [(info[1], info[2]) for info in outliers_info]
outlier_indices = [info[0] for info in outliers_info]

# Display the IDs and corresponding outlier values
print("Outliers (ID, Temperature):", outliers)
observation_id = outliers[0][0]

#### Update Observation outlier

In [ ]:
new_result = (results[outlier_indices[0] - 1] + results[outlier_indices[0] + 1]) / 2

body = {
    "result": new_result,
    "resultQuality": "100",
    "Commit": {
        "author": "Your Name",
        "message": "update result",
    }
}

# POST request with the JSON body
response = requests.patch(base_url + f"Observations({observation_id})", data=json.dumps(body))

# Check if the request was successful (status code 2xx)
if response.status_code == 200:
    print(f"Observation {observation_id} updated successfully")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

datetime_update = datetime.now()

#### Retrieve Observation outlier after update

In [ ]:
response = requests.get(f"{base_url}Observations({observation_id})?$expand=Commit")
json_data = json.dumps(response.json(), indent=2)
md = f"```json\n{json_data}\n```"
display(Markdown(md))

#### Retrieve Observation outlier at a specific instant (with the as_of value set to one second prior to the update date)

In [ ]:
datetime_before_update = datetime_update - timedelta(seconds=1)
datetime_before_update = datetime_before_update.strftime("%Y-%m-%dT%H:%M:%S.%f")
datetime_update = datetime_update.strftime("%Y-%m-%dT%H:%M:%S.%f")
response = requests.get(f"{base_url}Observations({observation_id})?$expand=Commit&$as_of={datetime_before_update}")
json_data = json.dumps(response.json(), indent=2)
md = f"```json\n{json_data}\n```"
display(Markdown(md))

#### Plot Observations after update

In [ ]:
response = requests.get(f"{base_url}Thing({thing_id})/Datastreams/Observations")
json_data = json.dumps(response.json(), indent=2)

# Parse the JSON data
data = json.loads(json_data)

# Extract the phenomenon times and results
phenomenon_times = [datetime.strptime(obs["phenomenonTime"], "%Y-%m-%dT%H:%M:%S%z") for obs in data["value"]]
results = [obs["result"] for obs in data["value"]]

# Plotting the results
plt.figure(figsize=(12, 7))  # Increase figure size for better visibility

# Customize the plot with better aesthetics
plt.plot(phenomenon_times, results, marker='o', markersize=8, linestyle='-', color='#1D4E89', label="Water Temperature", linewidth=2)

# Formatting the x-axis to show dates and times properly
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=1))
plt.gcf().autofmt_xdate()

# Adding a grid with better styling
plt.grid(True, which='both', linestyle='--', linewidth=0.6, color='gray', alpha=0.7)

# Adding labels and title with enhanced fonts
plt.title('Water Temperature Over Time', fontsize=16, fontweight='bold', color='#264653')
plt.xlabel('Phenomenon Time', fontsize=14, fontweight='bold', color='#264653')
plt.ylabel('Temperature (°C)', fontsize=14, fontweight='bold', color='#264653')

# Adding a legend
plt.legend(loc='upper left', fontsize=12)

# Adding custom ticks on the y-axis
plt.yticks(fontsize=12)

# Tight layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()

#### Retrieve Observation outlier within a time interval (between the date prior to the update and the date following it)

In [ ]:
response = requests.get(f"{base_url}Observations?$filter=id eq {observation_id}&$from_to={datetime_before_update}, {datetime_update}")
json_data = json.dumps(response.json(), indent=2)
md = f"```json\n{json_data}\n```"
display(Markdown(md))